<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd
import spacy
import re

In [2]:
yelp = pd.read_json('./data/review_sample.json', lines=True)
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [3]:
nlp = spacy.load('en_core_web_lg')
STOPWORDS = nlp.Defaults.stop_words.union({' ', ''})

In [4]:
def tokenize(doc):
    '''
    function to tokenize, remove stop words, and return list of tokens from an input doc
    '''
    text = re.sub(r'[^a-zA-Z ]', '', doc)
    text = text.lower()
    tokenizer = spacy.tokenizer.Tokenizer(nlp.vocab)
    tokens = tokenizer(text)
    list_of_tokens = [t for t in tokens if (str(t) not in STOPWORDS) and (t.is_punct == False)]
    return (list_of_tokens)

In [5]:
# test to make sure common stop words are removed and input is correct format
a_str = 'hi, this is a TEST string to check if## the toKenize function! @#$@#works $@//'
tokenize(a_str)

[hi, test, string, check, tokenize, function, works]

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [7]:
df = yelp.copy()
df.head(1)

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w


In [8]:
# creates a token feature
df['tokens'] = df['text'].apply(tokenize)
df['tokens'].head()

0    [beware, fake, fake, fakewe, small, business, ...
1    [came, lunch, togo, service, quick, staff, fri...
2    [ive, vegas, dozens, times, stepped, foot, cir...
3    [went, night, closed, street, party, best, act...
4    [starsnot, bad, price, lunch, seniors, pay, ea...
Name: tokens, dtype: object

In [9]:
tfidf = TfidfVectorizer(stop_words=STOPWORDS, min_df = 0.01, max_df=0.95, lowercase=True)
sparse_dtm = tfidf.fit_transform(df['text'])

In [10]:
dtm = pd.DataFrame(sparse_dtm.todense(), columns=tfidf.get_feature_names())
dtm.head()

,00,10,100,11,12,15,20,25,30,40,...,wouldn,wow,wrong,year,years,yelp,yes,young,yum,yummy
0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.166455,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.146159,0.0,0.0,...,0.0,0.0,0.0,0.124542,0.0,0.0,0.0,0.156979,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.205986,0.184545,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0


In [11]:
nn = NearestNeighbors(n_neighbors=10, algorithm='ball_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=10, p=2, radius=1.0)

In [12]:
fake_review = 'never go here, food bad, service bad, staff bad, I had to order more than once, please just save yourself the time'
fake_vec = tfidf.transform([fake_review])

In [13]:
sim_reviews = nn.kneighbors(fake_vec.todense())

In [14]:
# all reviews are complaining about how bad the place was except for one that is saying that it was never bad but uses the same sort of words as a bad review
for review in sim_reviews[1][0]:
    print(df['text'][review])
    print('')

only 1 word describes this place............. yuck!
bad food, bad beer, bad service, bad atmosphere...............

Don't waste your money. Bad management!!!  Bad service. Food over priced!!!

O  o  thenk 6nnn  .b  cgv  xx TV cvg  9 nvehxcfvvv3c nb b  c  y4  nb and the vghvhridd h 0d  c       v   3,  vv  4  ruddy

天氣很熱吃不下東西，今天我點了一個韓國冷面湯、餐後點了甜點，冰沙系列不會太甜膩，覺得店家很用心製作，包含擺盤精緻、佐料衛生，夏日想開胃，這是一個不錯的選擇，服務人員也很敬業，以後會常常來

Bon massage.  Spa propre organisé manque juste le stationnement prix abordable en promotion personnels respectueux

旅行でラスベガスに来ましたがネイルがはげてるのが気になり、探したお店でした。
質問にも丁寧に答えてくれましたし、日本人の方も日本語が話せる方も居て、とても綺麗で居心地のいいお店でした。 
ネイルはちはるさんと言う綺麗な方が丁寧にしてくれとても気に入りました。
予定になかったまつ毛エクステもお願いし、日本ではまだあまりないブラウンカラーのエクステをしてもらい、とても気に入りました。
また是非マッサージなどで伺いたいと思います。

Been eating here since they opened and I give it 5 stars because of consistency. I have never went there and got bad service or bad food. Everything on the menu is good and if you are in a hurry they have a drive thru. Do yourself a favor and try this place

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [15]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD

In [16]:
vectorizor = TfidfVectorizer(stop_words='english', lowercase=True, min_df=0.1, max_df=0.90)

classifier = GradientBoostingClassifier(n_estimators=100)

svd = TruncatedSVD(n_iter=100, algorithm='randomized')

In [17]:
lsi = Pipeline([('vect', vectorizor), ('svd', svd)])
pipe = Pipeline([('lsi', lsi), ('clf', classifier)])

params = {'lsi__svd__n_components': (10,15,20),
          'lsi__vect__max_features': ( 500, 650, 800)
}

In [18]:
grid_search = GridSearchCV(pipe, params, cv=5, n_jobs=-1)
grid_search.fit(df['text'], df['stars'])
print('best Score: ', grid_search.best_score_)
grid_search.best_params_

best Score:  0.5057


{'lsi__svd__n_components': 20, 'lsi__vect__max_features': 650}

In [19]:
mod = grid_search.best_estimator_
prediction_rating = mod.predict([fake_review])[0]
print(f'My fake review was predicted to give a {prediction_rating} star rating.')

My fake review was predicted to give a 1 star rating.


## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [20]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess

In [21]:
def gensim_tokenize(text):
    '''
    error when using dictionary on spacy tokens, using this function instead for this section
    '''
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

df['gen_tokens'] = df['text'].apply(gensim_tokenize)

Learn the vocubalary of the yelp data:

In [22]:
id2word = Dictionary(df['gen_tokens'])

Create a bag of words representation of the entire corpus

In [23]:
corpus = [id2word.doc2bow(tok) for tok in df['gen_tokens']]

Your LDA model should be ready for estimation: 

In [24]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 6
                  )

Create 1-2 visualizations of the results

In [40]:
lda_values = [lda[x] for x in corpus]

word_list = [re.findall(r'"([^"]*)"',t[1])[:3] for t in lda.print_topics()]

topics = []
for words in word_list:
    topic = ' '.join(words)
    topics.append(topic)   

def update(doc):
        d_dist = {k:0 for k in range(0,6)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in lda_values]

DF2 = pd.DataFrame(new_distro)
DF2.columns = topics
DF2.head()

,food good place,great good place,place food like,service food great,good food place,food great place
0,0.127980,0.143970,0.206559,0.264756,0.148257,0.108478
1,0.213047,0.099833,0.070011,0.103509,0.254689,0.258911
2,0.131975,0.203868,0.110107,0.137670,0.186483,0.229896
3,0.129162,0.150615,0.251733,0.186775,0.198023,0.083692
4,0.289318,0.195200,0.081343,0.158406,0.193659,0.082073


In [41]:
import warnings
warnings.filterwarnings('ignore')

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.001638 -0.003892       1        1  17.104204
5      0.001774  0.002806       2        1  16.950991
0      0.003938 -0.001016       3        1  16.792339
2     -0.004495 -0.002717       4        1  16.516380
1     -0.000636  0.002446       5        1  16.404606
3     -0.002219  0.002373       6        1  16.231476, topic_info=     Category         Freq        Term        Total  loglift  logprob
45    Default  3499.000000     service  3499.000000  30.0000  30.0000
210   Default  4132.000000       great  4132.000000  29.0000  29.0000
40    Default  4459.000000        good  4459.000000  28.0000  28.0000
193   Default  1316.000000     amazing  1316.000000  27.0000  27.0000
360   Default  1398.000000  definitely  1398.000000  26.0000  26.0000
92    Default  1689.000000        nice  1689.000000  25.0000  25.0000
145   Default  4755.000000        food  4755.000000  24.0000  24.0000
949   Default   699.000000       happy   699.000000  23.0000  23.0000
111   Default   826.000000       times   826.000000  22.0000  22.0000
446   Default  1169.000000         day  1169.000000  21.0000  21.0000
117   Default  1432.000000        went  1432.000000  20.0000  20.0000
743   Default  1592.000000        love  1592.000000  19.0000  19.0000
122   Default  1752.000000        best  1752.000000  18.0000  18.0000
391   Default  1450.000000       order  1450.000000  17.0000  17.0000
158   Default   873.000000       price   873.000000  16.0000  16.0000
185   Default  1123.000000   recommend  1123.000000  15.0000  15.0000
1075  Default  1119.000000         new  1119.000000  14.0000  14.0000
138   Default   804.000000         bit   804.000000  13.0000  13.0000
37    Default  1445.000000        came  1445.000000  12.0000  12.0000
34    Default  3304.000000        time  3304.000000  11.0000  11.0000
541   Default   909.000000        work   909.000000  10.0000  10.0000
30    Default  1163.000000        said  1163.000000   9.0000   9.0000
969   Default   392.000000   fantastic   392.000000   8.0000   8.0000
46    Default  1508.000000       staff  1508.000000   7.0000   7.0000
375   Default   908.000000       fresh   908.000000   6.0000   6.0000
417   Default   883.000000        took   883.000000   5.0000   5.0000
619   Default   357.000000       thank   357.000000   4.0000   4.0000
266   Default  1251.000000  experience  1251.000000   3.0000   3.0000
353   Default  1403.000000        come  1403.000000   2.0000   2.0000
108   Default   922.000000        sure   922.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
131    Topic6   272.218719      people  1382.892456   0.1929  -5.6719
687    Topic6   134.903976   excellent   644.791016   0.2539  -6.3739
945    Topic6   143.746490       clean   700.532104   0.2344  -6.3104
743    Topic6   297.792023        love  1592.274902   0.1417  -5.5821
18     Topic6   572.430847        like  3334.742920   0.0560  -4.9286
201    Topic6   325.349823         don  1795.681641   0.1100  -5.4936
34     Topic6   554.865295        time  3304.222412   0.0340  -4.9598
210    Topic6   668.315002       great  4132.827637  -0.0037  -4.7737
37     Topic6   267.070099        came  1445.427490   0.1296  -5.6910
469    Topic6    93.368675      having   438.651550   0.2711  -6.7419
137    Topic6   187.088638         bar   983.790833   0.1584  -6.0469
266    Topic6   227.853348  experience  1251.177002   0.1151  -5.8498
75     Topic6   336.625732         got  1987.676514   0.0425  -5.4595
145    Topic6   687.551208        food  4755.264160  -0.1157  -4.7454
36     Topic6   178.758286        wait   942.685303   0.1555  -6.0925
30     Topic6   209.276367        said  1163.274536   0.1029  -5.9348
133    Topic6   605.624756       place  4560.168945  -0.2006  -4.8722
40     Topic6   587.662048        good  4459.908203  -0.2085  -4.9023
431    Topic6   201.26582

In [49]:
# average star rating 
df['stars'].sum()/len(df)

3.7358

Looking at the dataframe of the topics, when labled with just three words it apears that they are all basicaly the same topic.
Some topics use the word good some use great so It can be infered that these topics would be the difference between 3-5 star reviews.
This makes sense seeing as the average star rating is 3.7. Most reviews in this range would say that they liked something but then
talk about what they did not like, resulting in the three most common words of each topic being simillar.

Upon looking at the LDAvis plot we can see the other words used in these topics to see greater variation.
Even though many of the words are the same, the frequency of use among topics is different. For example,
topic 1 and 4 both use many of the same words, but topic 4 has the word amazing in much higher frequency.
This could represent the difference in topics used among 4 and 5 star reviews. And upon lowering the relevence 
you can see words used such as yuk and rude, even though the most common words in those topics were "good food place".

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)